In [ ]:
import sys, os
sys.path.append(os.path.abspath("/home/vincnet/projects/ml_project/streamlined_pipeline/cancer_mutation_analysis/"))  # path to the folder that contains ml_large_data/

In [ ]:
import json, numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, confusion_matrix
)
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import KNNImputer

import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from numpy.random import default_rng

from ml_large_data.baselines import make_baseline_models
from ml_large_data.metrics import compute_all_metrics, find_best_threshold_mcc
from ml_large_data.utils import make_weights

#Config
N_BOOT    = 100        # number of bootstrap replicates (raise for tighter CIs)
CV_FOLDS  = 10         # stratified K-folds for tree learners inside each bootstrap
TREE_MODELS = {"LightGBM","XGBoost","CatBoost"}
RANDOM_STATE = 42

TRAIN_FILE   = "train_set_10.4k.csv" 
FEATURES_CSV = "feature_sets_large_study/pls_representative_features.csv"
HOLDOUT_FILE = "test_set_2.6k.csv"      

In [ ]:
# Load data       
feature_list = pd.read_csv(FEATURES_CSV)["feature"].astype(str).tolist()
df_train = pd.read_csv(TRAIN_FILE)
y_train = df_train["pathogenicity_label"].astype(int)
X_train_raw0 = df_train.drop(columns=["pathogenicity_label"])

# keep only columns present in both the file and feature list (selected features from the feature selection notebook)
keep_cols_train = [c for c in feature_list if c in X_train_raw0.columns]
missing_train   = sorted(set(feature_list) - set(keep_cols_train))
if missing_train:
    print(f"[Info] {len(missing_train)} features from list not found in TRAIN (e.g. {missing_train[:6]})")

X_train_raw = X_train_raw0.loc[:, keep_cols_train].copy()
pre = Pipeline([("imputer", KNNImputer(n_neighbors=5, metric="nan_euclidean"))])

df_test     = pd.read_csv(HOLDOUT_FILE)
y_test      = df_test["pathogenicity_label"].astype(int)
X_test_raw0 = df_test.drop(columns=["pathogenicity_label"])

keep_cols_test = [c for c in feature_list if c in X_test_raw0.columns]
missing_test   = sorted(set(feature_list) - set(keep_cols_test))
if missing_test:
    print(f"[Info] {len(missing_test)} features from list not found in TEST (e.g. {missing_test[:6]})")

X_test_raw = X_test_raw0.loc[:, keep_cols_test].copy()

# align TEST columns to TRAIN columns (order + any missing become NaN -> imputer handles)
X_test_raw = X_test_raw.reindex(columns=X_train_raw.columns)

In [5]:
# Merge TRAIN + TEST once
X_all_raw = pd.concat([X_train_raw, X_test_raw], axis=0, ignore_index=True)
y_all     = pd.concat([y_train, y_test], axis=0, ignore_index=True).astype(int)
assert list(X_all_raw.columns) == list(X_train_raw.columns)

In [ ]:
# Load best params from JSON
with open("best_params.json") as f:
    best = json.load(f)

# Build frozen pipelines
def build_tree_pipes(pre):
    # LightGBM
    lgb_params = {
        **best["LightGBM"]["best_params"],
        "objective":"binary","metric":"auc","verbosity":-1,"boosting_type":"gbdt",
        "n_estimators":1000,"random_state":RANDOM_STATE
    }
    lgb_pipe = Pipeline([("pre", clone(pre)), ("clf", LGBMClassifier(**lgb_params))])

    # XGBoost
    xgb_params = {**best["XGBoost"]["best_params"], "n_estimators":1000, "random_state":RANDOM_STATE}
    xgb_pipe = Pipeline([("pre", clone(pre)), ("clf", xgb.XGBClassifier(**xgb_params))])

    # CatBoost
    cb_params  = {**best["CatBoost"]["best_params"], "random_state":RANDOM_STATE, "verbose":0}
    cb_pipe = Pipeline([("pre", clone(pre)), ("clf", CatBoostClassifier(**cb_params))])

    return {"LightGBM": lgb_pipe, "XGBoost": xgb_pipe, "CatBoost": cb_pipe}

# Baselines (use same preprocessor up front)
baseline_pipes = {name: Pipeline([("pre", clone(pre)), ("clf", mdl)])
                  for name, mdl in make_baseline_models().items()}

all_models = {}
all_models.update(baseline_pipes)
all_models.update(build_tree_pipes(pre))

# Helper: interpolate curve (for mean + CI later)
def _interp_curve(xs, ys, grid):
    order = np.argsort(xs)
    xs, ys = xs[order], ys[order]
    uniq, idx = np.unique(xs, return_index=True)
    return np.interp(grid, uniq, ys[idx], left=ys[idx[0]], right=ys[idx[-1]])

def cv_predict_on_oob(pipe_template, X_boot, y_boot, X_oob, *,
                      folds=10, seed=42, model_name="", boot_idx=None, total_boot=None):
    """Stratified K-fold fit on bootstrap sample; average predicted proba on OOB."""
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    proba_sum = np.zeros(len(X_oob), dtype=float)
    for k, (tr_idx, _) in enumerate(skf.split(X_boot, y_boot), start=1):
        # Progress line: Model | Bootstrap i/N | Fold k/K
        if boot_idx is not None and total_boot is not None:
            print(f"\r{model_name} | Bootstrap {boot_idx}/{total_boot} | Fold {k}/{folds}   ",
                  end="", flush=True)
        pipe_fold = clone(pipe_template)
        _, sw_tr, _ = make_weights(y_boot.iloc[tr_idx])
        pipe_fold.fit(
            X_boot.iloc[tr_idx],
            y_boot.iloc[tr_idx],
            **{"clf__sample_weight": sw_tr}
        )
        proba_sum += pipe_fold.predict_proba(X_oob)[:, 1]
    # newline after the folds of this bootstrap
    print()
    return proba_sum / folds

rng = default_rng(0)
n = len(X_all_raw)

boot_results = {}

# Main bootstrap loop
for name, pipe in all_models.items():
    print(f"Bootstrapping (OOB) — {name}")
    roc_aucs, pr_aucs = [], []
    metrics_list, cms = [], []
    roc_grid = np.linspace(0, 1, 101)
    pr_grid  = np.linspace(0, 1, 101)
    roc_tprs, pr_precs = [], []

    for b in range(N_BOOT):
        # sample with replacement
        print(f" Bootstrap {b+1}/{N_BOOT}", end="\r")
        idx_boot = rng.integers(0, n, size=n)
        mask = np.zeros(n, dtype=bool); mask[idx_boot] = True
        idx_oob = np.where(~mask)[0]
        if idx_oob.size < 2:
            continue

        y_oob = y_all.iloc[idx_oob]
        if y_oob.nunique() < 2:
            continue  # AUC undefined if only one class in OOB

        X_boot = X_all_raw.iloc[idx_boot]; y_boot = y_all.iloc[idx_boot]
        X_oob  = X_all_raw.iloc[idx_oob]

        # Tree learners: Stratified 10CV inside bootstrap. Linear models and BNN: single fit
        if name in TREE_MODELS:
            p = cv_predict_on_oob(
                pipe, X_boot, y_boot, X_oob,
                folds=CV_FOLDS, seed=RANDOM_STATE,
                model_name=name, boot_idx=b+1, total_boot=N_BOOT
            )
        else:
            pipe_b = clone(pipe)
            # try to pass sample_weight if model supports it
            try:
                _, sw_tr, _ = make_weights(y_boot)
                pipe_b.fit(X_boot, y_boot, **{"clf__sample_weight": sw_tr})
            except Exception:
                pipe_b.fit(X_boot, y_boot)
            p = pipe_b.predict_proba(X_oob)[:, 1]

        thr, _ = find_best_threshold_mcc(y_oob, p)
        yhat = (p >= thr).astype(int)

        # scalar metrics
        m = compute_all_metrics(y_oob, yhat, p)
        metrics_list.append(m)

        # curves + areas
        fpr, tpr, _ = roc_curve(y_oob, p)
        precision, recall, _ = precision_recall_curve(y_oob, p)

        roc_aucs.append(auc(fpr, tpr))
        pr_aucs.append(auc(recall, precision))
        roc_tprs.append(_interp_curve(fpr, tpr, roc_grid))
        pr_precs.append(_interp_curve(recall, precision, pr_grid))

        # confusion matrix
        cm = confusion_matrix(y_oob, yhat, labels=[0,1])  # [[TN,FP],[FN,TP]]
        cms.append(cm)

    boot_results[name] = {
        "metrics": pd.DataFrame(metrics_list),
        "roc_auc": np.array(roc_aucs),
        "pr_auc":  np.array(pr_aucs),
        "roc_grid": roc_grid,
        "roc_tprs": np.vstack(roc_tprs) if roc_tprs else np.empty((0, len(roc_grid))),
        "pr_grid": pr_grid,
        "pr_precs": np.vstack(pr_precs) if pr_precs else np.empty((0, len(pr_grid))),
        "cms": np.array(cms)
    }

# Summary table (mean + 95% CI)
def mean_ci(a):
    a = np.asarray(a, float)
    return np.nanmean(a), np.nanpercentile(a, 2.5), np.nanpercentile(a, 97.5)

rows = []
for name, res in boot_results.items():
    if res["metrics"].empty: 
        continue
    M = res["metrics"].copy()
    M["ROC AUC"] = res["roc_auc"]
    M["PR AUC"]  = res["pr_auc"]
    out = {"Model": name}
    for col in ["ROC AUC","PR AUC","MCC","F1 Score","Balanced Accuracy","Precision","Recall"]:
        mu, lo, hi = mean_ci(M[col].values)
        out[col] = f"{mu:.3f} [{lo:.3f}, {hi:.3f}]"
    rows.append(out)

summary_df = pd.DataFrame(rows).set_index("Model").sort_values("ROC AUC")
display(summary_df)

# Plots with mean + 95% CI from bootstrap
def plot_roc_ci(res, title):
    grid, T = res["roc_grid"], res["roc_tprs"]
    if T.size == 0: 
        print("No valid ROC curves."); return
    mean_tpr = T.mean(axis=0)
    lo, hi = np.percentile(T, 2.5, axis=0), np.percentile(T, 97.5, axis=0)
    mean_auc = res["roc_auc"].mean()
    lo_auc, hi_auc = np.percentile(res["roc_auc"], 2.5), np.percentile(res["roc_auc"], 97.5)

    plt.figure(figsize=(7,6))
    plt.plot(grid, mean_tpr, lw=2, label=f"Mean ROC (AUC {mean_auc:.3f} [{lo_auc:.3f}, {hi_auc:.3f}])")
    plt.fill_between(grid, lo, hi, alpha=0.2, label="95% CI")
    plt.plot([0,1],[0,1],"k--", lw=1)
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

def plot_pr_ci(res, title):
    grid, P = res["pr_grid"], res["pr_precs"]
    if P.size == 0: 
        print("No valid PR curves."); return
    mean_prec = P.mean(axis=0)
    lo, hi = np.percentile(P, 2.5, axis=0), np.percentile(P, 97.5, axis=0)
    mean_auc = res["pr_auc"].mean()
    lo_auc, hi_auc = np.percentile(res["pr_auc"], 2.5), np.percentile(res["pr_auc"], 97.5)

    plt.figure(figsize=(7,6))
    plt.plot(grid, mean_prec, lw=2, label=f"Mean PR (AUC {mean_auc:.3f} [{lo_auc:.3f}, {hi_auc:.3f}])")
    plt.fill_between(grid, lo, hi, alpha=0.2, label="95% CI")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(title); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

def plot_confusion_ci(cms, title, normalize=False):
    A = np.asarray(cms, float)  # [B,2,2]
    if normalize:
        A = A / A.sum(axis=(1,2), keepdims=True)  # rate version
    mean = A.mean(axis=0)
    lo   = np.percentile(A, 2.5, axis=0)
    hi   = np.percentile(A, 97.5, axis=0)

    plt.figure(figsize=(6,5))
    sns.heatmap(mean, annot=False, fmt=".2f", cmap="Blues",
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    for i in range(2):
        for j in range(2):
            txt = f"{mean[i,j]:.1f}\n[{lo[i,j]:.1f}, {hi[i,j]:.1f}]"
            plt.text(j+0.5, i+0.5, txt, ha='center', va='center', color='black', fontsize=11)
    plt.title(title + "\n(mean count with 95% CI)")
    plt.tight_layout(); plt.show()

for name, res in boot_results.items():
    print(f"\n=== {name} ===")
    plot_roc_ci(res, f"ROC — {name} (OOB bootstrap, mean ± 95% CI)")
    plot_pr_ci(res,  f"PR — {name} (OOB bootstrap, mean ± 95% CI)")
    plot_confusion_ci(res["cms"], f"Confusion Matrix — {name} (OOB bootstrap)", normalize=False)

Bootstrapping (OOB) — Logistic Regression


KeyboardInterrupt: 